In [1]:
import pandas as pd
from scipy import stats
import random
import math


In [23]:
df = pd.read_csv("Data/Data/HeightWeight20.csv")
print(df)

        Height      Weight
0   187.571423  109.720985
1   174.706036   73.622732
2   188.239668   96.497550
3   182.196685   99.809504
4   177.499762   93.598619
5   170.822660   69.042216
6   174.714106   83.428219
7   173.605229   76.190352
8   170.228132   79.800187
9   161.179495   70.941642
10  160.475926   64.077173
11  159.097154   46.653553
12  157.677693   62.909437
13  152.477302   44.310238
14  150.109555   50.135381
15  168.078537   62.041159
16  170.350573   77.504315
17  162.224700   58.275377
18  175.346978   74.322166
19  157.338384   51.550324


In [43]:
def split_data(df : pd.DataFrame, seed : int, train_frac : float = 0.5):

    separator_index = math.floor(df.shape[0]*train_frac)

    sample_df = df.sample(frac=1.0, random_state=seed, ignore_index=True)
    train_df = sample_df.iloc[0:separator_index]
    test_df = sample_df.iloc[separator_index:].reset_index(drop=True)
    
    return(train_df, test_df)

def CalculateMSE(x_positions, y_positions, w0, w1):
    n = len(x_positions)
    MSE = 0
    for i in range(0, n):
        MSE += (w0 + w1*x_positions[i] - y_positions[i]) ** 2
    MSE *= 1/(2*n)
    return MSE

"""def split_data_to_fold(df : pd.DataFrame, seed : int, folds : int):

    data_per_fold = df.shape[0] / folds

    sample_df = df.sample(frac=1.0, random_state=seed, ignore_index=True)
    dfs = []
    remainder = 0
    add_lower, add_upper = 0, 0

    for i in range(folds-1):

        if remainder >= 1:
            remainder = 0
            add_lower = 1

        remainder += data_per_fold

        if remainder >= 1:
            add_upper = 1

        dfs.append(sample_df.iloc[i * math.floor(data_per_fold) + add_lower: (i+1) * math.floor(data_per_fold) + add_upper])

        if i < folds-2:
            add_lower, add_upper = 0, 0
        
    dfs.append(sample_df.iloc[(folds-1)*math.floor(data_per_fold) + add_upper:])

    return dfs"""

'def split_data_to_fold(df : pd.DataFrame, seed : int, folds : int):\n\n    data_per_fold = df.shape[0] / folds\n\n    sample_df = df.sample(frac=1.0, random_state=seed, ignore_index=True)\n    dfs = []\n    remainder = 0\n    add_lower, add_upper = 0, 0\n\n    for i in range(folds-1):\n\n        if remainder >= 1:\n            remainder = 0\n            add_lower = 1\n\n        remainder += data_per_fold\n\n        if remainder >= 1:\n            add_upper = 1\n\n        dfs.append(sample_df.iloc[i * math.floor(data_per_fold) + add_lower: (i+1) * math.floor(data_per_fold) + add_upper])\n\n        if i < folds-2:\n            add_lower, add_upper = 0, 0\n        \n    dfs.append(sample_df.iloc[(folds-1)*math.floor(data_per_fold) + add_upper:])\n\n    return dfs'

Holdout

In [67]:
train_df, test_df = split_data(df, 11010, 0.5)
print(train_df)
print(test_df)

       Height      Weight
0  170.822660   69.042216
1  152.477302   44.310238
2  174.714106   83.428219
3  174.706036   73.622732
4  170.350573   77.504315
5  160.475926   64.077173
6  173.605229   76.190352
7  187.571423  109.720985
8  162.224700   58.275377
9  150.109555   50.135381
       Height     Weight
0  182.196685  99.809504
1  168.078537  62.041159
2  175.346978  74.322166
3  170.228132  79.800187
4  188.239668  96.497550
5  157.338384  51.550324
6  161.179495  70.941642
7  157.677693  62.909437
8  177.499762  93.598619
9  159.097154  46.653553


In [68]:
slope, intercept, r, p, se = stats.linregress(train_df["Height"], train_df["Weight"])
print(slope, intercept)

1.5493022523574467 -189.19619910355522


In [69]:
print(CalculateMSE(test_df["Height"], test_df["Weight"], intercept, slope)**(1/2))

5.529377132393046


In [72]:
for i in range(10):
    train_df, test_df = split_data(df, i*10, 0.5)
    slope, intercept, r, p, se = stats.linregress(train_df["Height"], train_df["Weight"])
    print(CalculateMSE(test_df["Height"], test_df["Weight"], intercept, slope)**(1/2))

5.652096142953079
5.7897029042491726
5.53866332907573
5.701419482836353
6.871695790767007
5.2036002668618435
5.256019340640801
4.622999237427191
5.038353353501412
4.983975798078005
